In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tbp.monty.frameworks.utils.logging_utils import (load_stats,
                                                        print_overall_stats,)

In [ ]:
pretrain_path = os.path.expanduser("~/tbp/results/monty/pretrained_models/pretrained_ycb_v3/")
pretrained_dict = pretrain_path + "touch_1lm_10distinctobj/pretrained/"
pretrained_dict_5lms = pretrain_path + "supervised_pre_training_5lms_10distobj/pretrained/"

log_path = os.path.expanduser("~/tbp/results/monty/projects/evidence_eval_runs/logs/")
log_path_lambda = os.path.expanduser("~/tbp/results/monty/projects/evidence_eval_runs/lambda/")

exp_name_5lms_77obj = "five_lm_77obj_evidence_profiled/"
# exp_name_5lms_77obj = "randrot_noise_77obj_5lms_vision"
exp_name_5lms = "randrot_noise_10distobj_5lms_vision"
exp_name_1lm = "randrot_noise_10distobj_touch"


In [ ]:
_, eval_stats_5lms_77obj, _, lm_models_5lms = load_stats(log_path + exp_name_5lms_77obj,
                                                                load_train=False,
                                                                load_eval=True,
                                                                load_detailed=False,
                                                               load_models=False,
                                                                pretrained_dict=pretrained_dict_5lms,
                                                               )

_, eval_stats_5lms_77obj_lambda, _, lm_models_5lms = load_stats(log_path_lambda + exp_name_5lms_77obj,
                                                                load_train=False,
                                                                load_eval=True,
                                                                load_detailed=False,
                                                              load_models=False,
                                                                pretrained_dict=pretrained_dict_5lms,
                                                               )
_, eval_stats_5lms, _, lm_models_5lms = load_stats(log_path + exp_name_5lms,
                                                                load_train=False,
                                                                load_eval=True,
                                                                load_detailed=False,
                                                               load_models=False,
                                                                pretrained_dict=pretrained_dict_5lms,
                                                               )

_, eval_stats_5lms_lambda, _, lm_models_5lms = load_stats(log_path_lambda + exp_name_5lms,
                                                                load_train=False,
                                                                load_eval=True,
                                                                load_detailed=False,
                                                              load_models=False,
                                                                pretrained_dict=pretrained_dict_5lms,
                                                               )

In [ ]:
def print_5lm_stats(stats):
    majority_correct = 0
    step_times = []
    num_steps = []
    for episode in range(len(stats) // 5):
        episode_performance = stats['performance'][episode * 5:(episode+1) *5]
        if sum((episode_performance == 'correct') +
              (episode_performance == 'correct_mlh')) >= 3:
            majority_correct += 1
        episode_time = stats['time'][episode * 5]
        steps = stats['monty_matching_steps'][episode * 5]
        num_steps.append(steps)
        step_times.append(episode_time/steps)


    print(f"Percent Correct Episodes: {np.round(majority_correct/(len(stats) // 5)*100,3)}")
    print(f"Average time per step: {np.round(np.mean(step_times),2)}s with {np.round(np.mean(num_steps),2)} avg steps per episode")

In [ ]:
print("Laptop:")
print_5lm_stats(eval_stats_5lms)
print("Lambda:")
print_5lm_stats(eval_stats_5lms_lambda)
print("--77 obj--")
print("Laptop:")
print_5lm_stats(eval_stats_5lms_77obj)
print("Lambda:")
print_5lm_stats(eval_stats_5lms_77obj_lambda)

In [ ]:
_, eval_stats, _, _ = load_stats(log_path + exp_name_1lm,
                                                                load_train=False,
                                                                load_eval=True,
                                                                load_detailed=False,
                                                               load_models=False,
                                                                pretrained_dict=pretrained_dict_5lms,
                                                               )

_, eval_stats_lambda, _, lm_models_5lms = load_stats(log_path_lambda + exp_name_1lm,
                                                                load_train=False,
                                                                load_eval=True,
                                                                load_detailed=False,
                                                              load_models=False,
                                                                pretrained_dict=pretrained_dict_5lms,
                                                               )

In [ ]:
print("Laptop:")
print_overall_stats(eval_stats)
print("Lambda:")
print_overall_stats(eval_stats_lambda)

In [ ]:
eval_stats

## Profiles

In [ ]:
laptop_5lm_77obj_profile = pd.read_csv(log_path + "five_lm_77obj_evidence_profiled/profile/profile-eval_epoch_0_episode_0.csv")
lambda_5lm_77obj_profile = pd.read_csv(log_path_lambda + "five_lm_77obj_evidence_profiled/profile/profile-eval_epoch_0_episode_0.csv")
lambda_5lm_77obj_nomt_profile = pd.read_csv(log_path_lambda + "randrot_noise_10distobj_5lms_vision_nomt_profiled/profile/profile-eval_epoch_0_episode_0.csv")

In [ ]:
lambda_5lm_77obj_nomt_profile

In [ ]:
laptop_5lm_77obj_profile

In [ ]:
print("LAMBDA - NO MULTITHREADING EPISODE STATS:")
for i in range(20):
    print(f"spent {np.round(lambda_5lm_77obj_nomt_profile['cumtime'][i],2)}s on\n {lambda_5lm_77obj_nomt_profile['func'][i]}")

In [ ]:
from tbp.monty.frameworks.utils.profile_utils import (
    bar_chart_cumtime,
    bar_chart_tottime,
    get_data_from_df,
    print_top_k_functions,
    sort_by_cumtime,
    sort_by_tottime
)

In [ ]:
%matplotlib notebook

In [ ]:

bar_chart_cumtime(lambda_5lm_77obj_nomt_profile, n_functions=15)

In [ ]:
lambda_5lm_77obj_nomt_profile

In [ ]:
list(lambda_5lm_77obj_nomt_profile['func'][:15])

In [ ]:
top_15_func_names = [
    'KDTree.query',
    'einsum',
    'get_node_features',
    'get_pose_evidence_matrix',
    'get_angles_for_all_hypotheses',
    'calculate_evidence_for_new_locations',
    'numpy reduce',
    'np.dot',
    'get_custom_distances',
    'draw_observation',
    'get_feature_evidence_for_all_nodes',
    'numpy _clip_dep_invoke_with_casting',
    'numpy implement_array_function',
    'linalg.norm',
    'add_object'
]

In [ ]:
plt.figure()
ax = plt.subplot(1,1,1)
plt.bar(np.linspace(0,14,15),lambda_5lm_77obj_nomt_profile['cumtime'][:15])
ax.spines[['right', 'top']].set_visible(False)
plt.xticks(np.linspace(0,14,15),top_15_func_names,rotation = 90)
plt.ylabel('cumtime (s)')
plt.title('Time Taken by Top 15 Functions')
plt.tight_layout()
plt.show()

In [ ]:
list(lambda_5lm_77obj_nomt_profile['func'][:15])

In [ ]:
lambda_5lm_77obj_nomt_profile_sort = lambda_5lm_77obj_nomt_profile.sort_values('tottime', ascending=False)

In [ ]:
lambda_5lm_77obj_nomt_profile_sort['func'][:25]

In [ ]:
top_25_tottime_func_names = [
    'run_episode',
    'run_episode_steps',
    'step',
    'matching_step',
    'step_learning_modules',
    'matching_step',
    'compute_possible_matches',
    'update_possible_matches',
    'update_evidence',
    'calculate_evidence_for_new_locations',
    'query',
    'KDTree.query',
    'get_pose_evidence_matrix',
    'numpy implement_array_function',
    'get_angles_for_all_hypotheses',
    'einsum',
    'einsum',
    'umath c_einsum',
    'get_node_features',
    'get_custom_distances',
    'vote',
    'numpy reduce',
    'np.dot',
    'norm',
    'linalg.norm'
]

In [ ]:
plt.figure()
ax = plt.subplot(1,1,1)
plt.bar(np.linspace(0,24,25),lambda_5lm_77obj_nomt_profile_sort['tottime'][:25])
ax.spines[['right', 'top']].set_visible(False)
plt.xticks(np.linspace(0,24,25),top_25_tottime_func_names,rotation = 90)
plt.ylabel('cumtime (s)')
plt.title('Total Time Taken by Top 25 Functions')
plt.tight_layout()
plt.show()